In [1]:
from src.helper_functions import xor_hex_strings, repeating_key_xor, xor_bytes, hamming_distance, output_repeated_block, has_repeated_blocks, find_block
from src.symmetric_encryption import is_ecb_mode
import src.aes128 as aes128
import src.convert as convert
import src.padding as padding
# 
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64
import secrets

## **Challenge 17: PKCS#7 padding validation**

In [2]:
# Assume that key is unknown but same for all encryptions
encryption_oracle_17_key = secrets.token_bytes(16)

In [3]:
def encryption_oracle_17():
    xxxx = [
        b"MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=",
        b"MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=",
        b"MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==",
        b"MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==",
        b"MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl",
        b"MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==",
        b"MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==",
        b"MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=",
        b"MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=",
        b"MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93",
    ]

    chose = secrets.choice(xxxx)
    plaintext = padding.apply_pkcs_7(base64.b64decode(chose), 16)
    IV = secrets.token_bytes(16)
    return aes128.cbc_encrypt(plaintext, encryption_oracle_17_key, IV), IV


def correct_padding_oracle(ciphertext, IV) -> bool:
    plaintext = aes128.cbc_decrypt(ciphertext, encryption_oracle_17_key, IV)
    try:
        padding.remove_pkcs_7(plaintext)
        return True
    except Exception:
        return False

In [4]:
cyphertext, IV = encryption_oracle_17()
correct_padding_oracle(cyphertext, IV)

True